# 開始安裝

In [ ]:
#20250828執行過
%pip install ipython-sql pandas sqlalchemy

In [ ]:
%load_ext sql

In [ ]:
import sqlite3
import pandas as pd

# 資料庫路徑（根據實際位置調整）
db_path = "../SQL/mysql/database/hi.db"

# 連接到 SQLite 資料庫
conn = sqlite3.connect(db_path)


# 練習區

## import內容和DB路經

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

## 查詢

In [ ]:
# 執行查詢並用 Pandas 顯示結果  
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

query = "SELECT * FROM Daily_report WHERE tag like '%sql%';"  
df = pd.read_sql(query, conn)  
df

## 插入單筆新資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

# 建立 DataFrame
df = pd.DataFrame({
    'content': ['測試文字寬度設定'],
    'reference': ['SQL/sqlite-python.ipynb'],
    'tag': ['ipython-sql/sql/'],
    'created_at': [pd.Timestamp.now()]
})

# # 插入資料
df.to_sql('Daily_report', conn, if_exists='append', index=False)

# 全域設定（會影響整個 kernel）  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 100)  
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 30)  
pd.set_option('display.precision', 3)  
pd.set_option('display.expand_frame_repr', True)  
pd.set_option('display.colheader_justify', 'right')
# 讀取並顯示最新插入的資料  
result = pd.read_sql_query("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 5", conn)  # 讀取最新一筆資料
print(result)  


# 關閉連線
conn.close()


In [ ]:
import pandas as pd
import sqlite3

# 建立連線
conn = sqlite3.connect('../SQL/mysql/database/hi.db')

# 讀取最新一筆資料
result = pd.read_sql_query("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 3", conn)

# 手動右對齊顯示
def right_align_display(df):
    # 獲取每列最大寬度
    col_widths = {col: max(df[col].astype(str).str.len().max(), len(col)) for col in df.columns}
    
    # 打印表頭
    header = " ".join(f"{col:<{col_widths[col]}}" for col in df.columns)
    print(header)
    print("-" * len(header))
    
    # 打印資料
    for _, row in df.iterrows():
        row_str = " ".join(f"{str(row[col]):<{col_widths[col]}}" for col in df.columns)
        print(row_str)

# 顯示
right_align_display(result)


In [17]:
import pandas as pd
import sqlite3
import textwrap

# 建立連線並讀取示範資料
conn = sqlite3.connect('../SQL/mysql/database/hi.db')
result = pd.read_sql_query("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 3", conn)
conn.close()

def wrap_cell_fixed_width(text, width):
    """Wrap text into lines with max length = width. Return list of lines (at least [''])."""
    if pd.isna(text):
        return ['']
    s = str(text)
    if width <= 0:
        return [s]
    lines = textwrap.wrap(s, width=width, replace_whitespace=False, drop_whitespace=False)
    return lines if lines else ['']

def table_display_fixed_width(df, col_widths=None, default_width=20, padding=2, align='left'):
    """
    Display DataFrame in terminal with fixed column widths and wrapping.
    - col_widths: dict mapping column name -> width (int). If None, all columns use default_width.
    - default_width: width used when column not in col_widths.
    - padding: spaces between columns.
    - align: 'left' or 'right' for cell alignment.
    """
    # 設定每欄寬度
    if col_widths is None:
        col_widths = {col: default_width for col in df.columns}
    else:
        # 確保所有欄位都有寬度設定
        for col in df.columns:
            if col not in col_widths:
                col_widths[col] = default_width

    pad = ' ' * padding

    # 將每個 cell 轉成換行後的 list（依固定寬度）
    wrapped = {}
    for col in df.columns:
        width = col_widths[col]
        wrapped[col] = [wrap_cell_fixed_width(v, width) for v in df[col].tolist()]

    # 計算每列的最大子行數
    row_line_counts = [max(len(wrapped[col][i]) for col in df.columns) for i in range(len(df))]

    # 格式化 cell（對齊並截斷或補空白到固定寬度）
    def format_cell(s, width):
        # 若長度超過 width，強制截斷（但理論上 wrap 已處理過長情況）
        if len(s) > width:
            s = s[:width]
        return s.rjust(width) if align == 'right' else s.ljust(width)

    # 列印表頭（欄名會被限制在固定寬度，過長會截斷）
    header_parts = [format_cell(str(col)[:col_widths[col]], col_widths[col]) for col in df.columns]
    header = pad.join(header_parts)
    print(header)
    print('-' * len(header))

    # 列印每一列資料，逐子行列印
    for i in range(len(df)):
        for line_idx in range(row_line_counts[i]):
            parts = []
            for col in df.columns:
                cell_lines = wrapped[col][i]
                sub = cell_lines[line_idx] if line_idx < len(cell_lines) else ''
                parts.append(format_cell(sub, col_widths[col]))
            print(pad.join(parts))
        # 可選：列間空一行（若不要可註解掉下一行）
        # print()

# 範例：手動指定每個欄位寬度
# 假設你的欄位為 ['id', 'content', 'tag', 'created_at']，自行調整寬度
example_col_widths = {
    'id': 6,
    'content': 50,
    'tag': 30,
    'created_at': 20,
}

# 若不確定欄名，先看看 result.columns
print("Columns:", list(result.columns))

# 呼叫顯示函式（若欄位與 example_col_widths 不同，請調整字典）
table_display_fixed_width(result, col_widths=example_col_widths, padding=2, align='left')


Columns: ['id', 'content', 'reference', 'tag', 'created_at']
id      content                                             reference             tag                             created_at          
--------------------------------------------------------------------------------------------------------------------------------------
10      測試文字寬度設定                                            SQL/sqlite-           ipython-sql/sql/                2025-08-29          
                                                            python.ipynb                                          00:19:12.031858     
9       學習如何使用 SQLite 與 Python 進行資料庫操作                      SQL/sqlite-           ipython-sql/sql/                2025-08-28 14:46:34 
                                                            python.ipynb                                                              
8       失敗=>使用jupyter寫sqlite语法，改用python連接db並插入資料            SQL/sqlite-           ipython-sql/sql/                2025-08-28 14:2

## 插入多筆資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

# 建立包含多筆資料的 DataFrame
df = pd.DataFrame({
    'content': [
        '使用jupyter寫sqlite语法', 
        '學習pandas插入sqlite', 
        'SQLite資料庫操作'
    ],
    'reference': [
        'SQL/sqlite-python.ipynb', 
        'pandas/sqlite-insert.ipynb', 
        'database/sqlite-tutorial.md'
    ],
    'tag': [
        'ipython-sql/sql/', 
        'pandas/database', 
        'tutorial/sqlite'
    ]
})

# 插入多筆資料
df.to_sql('Daily_report', conn, if_exists='append', index=False) # append表示新增資料，不覆蓋原有資料。index=False表示不插入DataFrame的索引欄位。

# 關閉連線
conn.close()


## 刪除單筆及多筆資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor() # 建立游標

# 單筆刪除  
cur.execute("DELETE FROM Daily_report WHERE id = ?", (12,))  
conn.commit()

# 多筆刪除
# cur.execute("DELETE FROM Daily_report WHERE id IN (?, ?)", (4, 5))
# conn.commit()

result = pd.read_sql_query("SELECT * FROM Daily_report", conn)  # 
print(result) 